# 特徴語分析スクリプト

このスクリプトはspaCyライブラリを使用して事前に解析したテキストデータに対して特徴語分析を行う。N-gram分析、単語リストの作成、Scattertextによる可視化などの機能が含まれる。

## 使用方法

### 解析データの読み込み

1. 解析済みのテキストデータ（.spacyファイル）とそれに対応するテキストファイル（.txt）を用意する。
2. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `main_directory`: 親ディレクトリへの相対パス  
 - `subdirectories`: 読み込む子ディレクトリ名のリスト  
 - `nlp_model`: 解析に利用したモデル名
 - `selected_file_names`: 特定のファイルのみを読み込む場合はここにtxtファイル名を追加（空リストの場合は全ファイルを読み込む）

### N-gram分析の実行

1. スクリプト内の「ユーザーが設定するパラメーター」セクションで以下の変数を設定する。
 - `pos_tags`: 出力結果に含める品詞のリスト
 - `max_ngram`: 出力結果に含める最大のN-gramの長さ
 - `top_n`: 上位N件の頻度を出力
 - `stop_words_list`: ストップワードのリスト
 - `include_space`: n-gramのトークン間にスペースを含めるかどうか
 - `original_output_file`: オリジナルのn-gramの出力ファイル名  
 - `lemma_output_file`: 基本形に変換したn-gramの出力ファイル名
2. スクリプトを実行すると、指定したファイル名でCSVファイルが出力される。

## 分析内容

### 1. N-gram分析

指定した品詞や最大N-gramの長さに基づいて、単語やN-gramの出現頻度を分析する。結果はCSVファイルとして出力される。

### 2. 単語リストの作成

解析済みのDocオブジェクトから単語リストを作成し、各単語の原形、品詞、出現回数を記録する。結果はCSVファイルとして出力される。

### 3. Scattertext分析 

指定した2つのテキスト群の特徴的な単語を可視化するScattertextプロットを作成する。結果はHTMLファイルとして出力される。

# **ライブラリのインポート(最初に実行)**

In [1]:
import os
import spacy
from collections import Counter, defaultdict
import pandas as pd
from spacy.tokens import DocBin
import scattertext as st

# **解析データの読み込み**
## ユーザーが設定するパラメーター

In [2]:
# 基本設定
main_directory = "processed_data"  # 親ディレクトリへの相対パス
subdirectories = ["sub1"]  # 読み込む子ディレクトリ名のリスト
nlp_model = "en_core_web_sm"  # 解析に利用したモデル名

# 特定のファイルのみを読み込む場合は以下のリストにtxtファイル名を追加(空リストの場合は全ファイルを読み込む)
selected_file_names = []

## 解析データの読み込みを実行

In [ ]:
# spaCyの言語モデルをロード
nlp = spacy.load(nlp_model)

# 解析結果を格納する辞書の初期化
docs_dict = {}
# フィルタリングされたDocオブジェクトを格納する辞書の初期化
filtered_docs = {}

# 指定されたサブディレクトリ内のファイルを読み込む
for subdir in subdirectories:
    directory = os.path.join(main_directory, subdir)
    
    # サブディレクトリ内のファイルを読み込む
    for filename in os.listdir(directory): 
        if filename.endswith(".spacy"):  # .spacyファイルならば
            spacy_path = os.path.join(directory, filename)
            txt_filename = filename.replace(".spacy", ".txt")  # .spaCyファイルに対応するテキストファイル名を作成
            txt_path = os.path.join(directory, txt_filename)  # .spacyファイルに対応するテキストファイルのパス
            
            # .spacyファイルからDocオブジェクトを読み込む
            doc_bin = DocBin().from_disk(spacy_path)  # DocBinオブジェクトを読み込む
            docs = list(doc_bin.get_docs(nlp.vocab))  # Docオブジェクトをリストに格納
            
            # 対応する.txtファイルからファイル名を読み込む
            if os.path.exists(txt_path):
                with open(txt_path, 'r', encoding='utf-8') as f:
                    file_names = [line.strip() for line in f.readlines()]  # ファイル名をリストに格納
                
                # Docオブジェクトとファイル名の対応を辞書に格納
                for doc, fname in zip(docs, file_names):
                    docs_dict[fname] = doc
            else:
                print(f"警告: {txt_filename} に対応するテキストファイルが存在しません。")

# ユーザーがファイル名を指定した場合、それに対応するDocオブジェクトをフィルタリング
if selected_file_names:
    missing_files = []  # 存在しないファイル名を格納するリスト
    for fname in selected_file_names:
        if fname in docs_dict:  # 指定されたファイル名が読み込んだデータに存在する場合
            filtered_docs[fname] = docs_dict[fname]
        else:
            missing_files.append(fname)
    
    if missing_files:
        print(f"警告: 次の指定されたファイルは存在しません: {', '.join(missing_files)}")
    if filtered_docs:
        print("指定された一部のデータが読み込まれました。")
    else:
        print("指定されたファイルに対応するデータが見つかりませんでした。")
else:
    filtered_docs = docs_dict
    print("全てのデータが読み込まれました。")
print(f"読み込まれたデータ数: {len(filtered_docs)}")
print(f"読み込まれたファイル名: {list(filtered_docs.keys())}")

# **N-gram分析**
## ユーザーが設定するパラメーター

In [14]:
pos_tags = ['NOUN', 'ADJ', 'ADV','VERB', 'PROPN']  # 出力結果に含める品詞のリスト
max_ngram = 5    # 出力結果に含める最大のN-gramの長さ
top_n = 1000     # 上位N件の頻度を出力
include_space = True  # n-gramのトークン間にスペースを含めるかどうか

# 単語の集計用のストップワードのリスト
word_stop_words_list = ["%", "-"]  
# N-gram用のストップワードのリスト
ngram_stop_words_list = ['。', '、', '.', ',', '!', '！', '?', '？', ':', ';', '-', '&', '%', "'", '"', '#', '(', ')', '[', ']', '{', '}', "　"]

original_output_file = "n-gram.csv"  # オリジナルのn-gramの出力ファイル名
lemma_output_file = "n-gram_lemma.csv"  # 基本形に変換したn-gramの出力ファイル名

## 特徴語分析の実行（オリジナル）

In [ ]:
def get_word_freq(doc, pos_tags, word_count, stop_words=None):
    """
    単語の頻度をカウントする関数。
    
    Args:
        doc (Doc): spaCyのDocオブジェクト。
        pos_tags (list): 出力結果に含める品詞のリスト。
        word_count (Counter): 単語の出現頻度を格納するCounterオブジェクト。
        stop_words (set, optional): 除外するストップワードのセット。デフォルトはNone。
        
    Returns:
        Counter: 単語の出現頻度を格納したCounterオブジェクト。
    """
    tokenized_texts = [token.text.lower() for token in doc if token.pos_ in pos_tags and token.text.lower() not in stop_words] # 処理中の品詞のトークンを取得(stop_wordsを除く)
    word_count.update(tokenized_texts)  # Counterオブジェクトにトークンを追加
    return word_count

def get_ngram_freq(doc, ngram, separator, ngram_count, stop_words=None):
    """
    N-gramの頻度をカウントする関数。
    
    Args:
        doc (Doc): spaCyのDocオブジェクト。
        ngram (int): N-gramの長さ。
        separator (str): N-gramのトークン間のセパレータ。
        ngram_count (Counter): N-gramの出現頻度を格納するCounterオブジェクト。
        stop_words (set, optional): 除外するストップワードのセット。デフォルトはNone。

    Returns:
        Counter: N-gramの出現頻度を格納したCounterオブジェクト。
    """
    tokenized_texts = [token.text.lower() for token in doc if token.text.lower() not in stop_words]  # 句読点と空白のトークンを除外(注意)
    if ngram == 1:
        ngram_count.update(tokenized_texts)  # 1-gramの場合はトークンをそのまま追加
    else:
        # N-gramのリストを作成( 例: 2-gramの場合、['word1 word2', 'word2 word3', ...])
        ngrams = [separator.join(tokenized_texts[i:i+ngram]) for i in range(len(tokenized_texts)-ngram+1)]
        ngram_count.update(ngrams) # CounterオブジェクトにN-gramを追加
    return ngram_count

word_stop_words = set(word_stop_words_list)  # 単語のストップワードをセットに変換
ngram_stop_words = set(ngram_stop_words_list) # N-gramのストップワードをセットに変換
word_counts = {pos_tag: Counter() for pos_tag in pos_tags}  # 品詞ごとの単語の出現頻度を格納する辞書の初期化(例: {'NOUN': Counter(), 'ADJ': Counter(), ...}
ngram_counts = {i: Counter() for i in range(1, max_ngram + 1)}  # N-gramの出力結果を格納する辞書の初期化(例: {1: Counter(), 2: Counter(), ...}
separator = ' ' if include_space else ''  # N-gramのトークン間のセパレータを設定
results = []  # 出力結果（単語の出現頻度とN-gramの出現頻度）を格納するリストの初期化

# フィルタリングされたDocオブジェクトに対して単語の頻度とN-gramの頻度をカウント
for doc in filtered_docs.values():
    for pos_tag in pos_tags: # 品詞ごとに単語の出現頻度をカウント
        get_word_freq(doc, [pos_tag], word_counts[pos_tag], word_stop_words)
    for i in range(1, max_ngram + 1):  # N-gramごとに出現頻度をカウント
        get_ngram_freq(doc, i, separator, ngram_counts[i], ngram_stop_words)

# 品詞ごとの単語の出現頻度をデータフレームに変換
for pos_tag, count in word_counts.items():
    df = pd.DataFrame(count.most_common(top_n), columns=[f'{pos_tag}_word', f'{pos_tag}_count'])
    results.append(df)   # 品詞ごとのdfをリストに追加

# N-gramの出現頻度をデータフレームに変換
for i, count in ngram_counts.items():               
    df = pd.DataFrame(count.most_common(top_n), columns=[f'{i}gram_word', f'{i}gram_count'])
    results.append(df)  # N-gramのdfをリストに追加

pd.concat(results, axis=1).to_csv(original_output_file, index=False)   # リストに格納されたデータフレームを結合してCSVファイルに保存
print(f"CSVファイルが保存されました。ファイル名: {original_output_file}")

## 特徴語分析の実行（基本形）

In [ ]:
def get_word_freq(doc, pos_tags, word_count, stop_words=None):
    """
    単語の頻度をカウントする関数。
    
    Args:
        doc (Doc): spaCyのDocオブジェクト。
        pos_tags (list): 出力結果に含める品詞のリスト。
        word_count (Counter): 単語の出現頻度を格納するCounterオブジェクト。
        stop_words (set, optional): 除外するストップワードのセット。デフォルトはNone。
        
    Returns:
        Counter: 単語の出現頻度を格納したCounterオブジェクト。
    """
    tokenized_texts = [token.lemma_.lower() for token in doc if token.pos_ in pos_tags and token.lemma_.lower() not in stop_words]  # lemmatizationを適用（句読点と空白を除外）
    word_count.update(tokenized_texts) 
    return word_count

def get_ngram_freq(doc, ngram, separator, ngram_count, stop_words=None):
    """
    N-gramの頻度をカウントする関数。
    
    Args:
        doc (Doc): spaCyのDocオブジェクト。
        ngram (int): N-gramの長さ。
        separator (str): N-gramのトークン間のセパレータ。
        ngram_count (Counter): N-gramの出現頻度を格納するCounterオブジェクト。
        stop_words (set, optional): 除外するストップワードのセット。デフォルトはNone。
        
    Returns:
        Counter: N-gramの出現頻度を格納したCounterオブジェクト。
    """
    tokenized_texts = [token.lemma_.lower() for token in doc if token.lemma_.lower() not in stop_words]  # lemmatizationを適用（句読点と空白を除外）
    if ngram == 1:
        ngram_count.update(tokenized_texts)
    else:
        ngrams = [separator.join(tokenized_texts[i:i+ngram]) for i in range(len(tokenized_texts)-ngram+1)]
        ngram_count.update(ngrams)
    return ngram_count

word_stop_words = set(word_stop_words_list)
ngram_stop_words = set(ngram_stop_words_list)
word_counts = {pos_tag: Counter() for pos_tag in pos_tags}      
ngram_counts = {i: Counter() for i in range(1, max_ngram + 1)}  
separator = ' ' if include_space else ''
results = []

for doc in filtered_docs.values():
    for pos_tag in pos_tags:
        get_word_freq(doc, [pos_tag], word_counts[pos_tag], word_stop_words)
    for i in range(1, max_ngram + 1):
        get_ngram_freq(doc, i, separator, ngram_counts[i], ngram_stop_words)

for pos_tag, count in word_counts.items():              
    df = pd.DataFrame(count.most_common(top_n), columns=[f'{pos_tag}_word', f'{pos_tag}_count'])
    results.append(df)                                   

for i, count in ngram_counts.items():               
    df = pd.DataFrame(count.most_common(top_n), columns=[f'{i}gram_word', f'{i}gram_count'])
    results.append(df)                               

pd.concat(results, axis=1).to_csv(lemma_output_file, index=False) 
print(f"CSVファイルが保存されました。ファイル名: {lemma_output_file}")

# **単語リストの作成**
## ユーザーが設定するパラメーター

In [ ]:
output_file = "word_list.csv"  # 出力ファイル名

## 単語リスト作成を実行

In [ ]:
def get_word_list_and_attributes(doc, word_list):
    """
    単語リストを作成し、各単語の属性を追加する関数。
    
    Args:
        doc (Doc): spaCyのDocオブジェクト。
        word_list (defaultdict): 単語リストを格納する辞書。
        
    Returns:
        defaultdict: 単語リストと各単語の属性を格納した辞書。
    """
    for token in doc:
        word_key = (token.text.lower(), token.pos_)          # 単語と品詞の組み合わせをキーとして用いる
        word_list[word_key]['token'] = token.text.lower()    # tokenを追加
        word_list[word_key]['lemma'] = token.lemma_.lower()  # lemmaを追加
        word_list[word_key]['pos'] = token.pos_              # posを追加
        word_list[word_key]['count'] += 1                    # countを1増やす
    return word_list

# 単語リストを格納する辞書の初期化(例: {('word', 'NOUN'): {'token': 'word', 'lemma': 'word', 'pos': 'NOUN', 'count': 1}, ...}
word_list = defaultdict(lambda: {'token': '', 'lemma': '', 'pos': '', 'count': 0})

for doc in filtered_docs.values():
    word_list = get_word_list_and_attributes(doc, word_list)

# 辞書の値だけを用いてDataFrameを作成
df_word_list = pd.DataFrame(list(word_list.values()))

# ソートと列の並び替え
df_word_list = df_word_list.sort_values(['token', 'pos'])
df_word_list = df_word_list[['token', 'lemma', 'pos', 'count']]

df_word_list.to_csv(output_file, index=False)
print(f"単語リストのCSVファイルが保存されました。ファイル名: {output_file}")

# **Scattertext**
## ユーザーが設定するパラメーター

In [ ]:
# グループ1とグループ2に含めるtxtファイル名のリスト
group1 = ['A.txt']  # y軸に表示するグループ1に含めるファイル名のリスト
group2 = ['B.txt']  # x軸に表示するグループ2に含めるファイル名のリスト

max_terms = 1000  # 表示する単語数
minimum_term_frequency = 2  # 最小出現頻度
pos_tags = ['PROPN', 'NOUN', 'ADJ', 'ADV']  # 出力結果に含める品詞のリスト
output_file_html = "scattertext_output.html" # 出力ファイル名

custom_stopwords = {}  # ストップワードを指定

## Scattertext作成の実行

In [ ]:
data = []  # Scattertextに渡すデータを格納するリストの初期化

# 品詞フィルタリングとストップワード除外を適用してテキストを前処理
for fname, doc in filtered_docs.items():
    if fname not in group1 and fname not in group2:  # グループに含めるファイル名でない場合はスキップ
        continue
    # テキストを前処理してリストに追加(品詞フィルタリングとストップワード除外を適用)
    filtered_text = ' '.join([token.lemma_.lower() for token in doc if token.pos_ in pos_tags and token.lemma_.lower() not in custom_stopwords])
    # ファイル名に応じてデータをグループに分けてリストに追加
    if fname in group1:
        data.append({'text': filtered_text, 'group': 'Group 1'})
    elif fname in group2:
        data.append({'text': filtered_text, 'group': 'Group 2'})

df = pd.DataFrame(data)

if df.empty:
    print("エラー: 選択されたファイルに対応するデータが見つかりませんでした。")
else:
    try:
        # Corpusオブジェクトを作成
        corpus = st.CorpusFromPandas(
            df,
            category_col='group',
            text_col='text',
            nlp=nlp
        ).build()

        # Scattertextを生成
        html = st.produce_scattertext_explorer(
            corpus,
            category='Group 1',  # y軸カテゴリ
            category_name='Group 1',  # y軸ラベル
            not_category_name='Group 2',  # x軸ラベル
            width_in_pixels=1000,  # 幅
            minimum_term_frequency=minimum_term_frequency, # 指定より出現回数の多い単語のみをプロット
            max_terms=max_terms  # 表示する単語数
        )

        with open(output_file_html, "w", encoding='utf-8') as file:
            file.write(html)
        print(f"Scattertextプロットが '{output_file_html}' に保存されました。")
    except Exception as e:
        print(f"エラー: Scattertextプロットの生成中にエラーが発生しました。詳細: {e}")